# Analyze the JS divergences

## Preanble

In [148]:
import psutil
p = psutil.Process()
p.cpu_affinity([0])

import numpy as np 
import matplotlib.pyplot as plt
import pickle
from IPython.display import display, Math
import pandas as pd

Utilities:

In [149]:
parameters_str = [r"$\mathcal{M}_c$",
                  r"$q$",
                  r"$\chi_1$",
                  r"$\chi_2$",
                  r"$\Lambda_1$",
                  r"$\Lambda_2$",
                  r"$d_L$",
                  r"$\phi_c$",
                  r"$\cos \iota$",
                  r"$\psi$",
                  r"$\alpha$",
                  r"$\sin \delta$",
]

In [150]:
get_path = lambda x: f"../figures/{x}_js_div.pkl"

In [151]:
def get_js_table(events_list: list[str],
                 nb_round: int = 6,
                 filler_str: str = "0") -> pd.DataFrame:
    """
    Function to create the JS divergence table by fetching the values from the respective run dirs.

    Args:
        events_list (list[str]): List of events to analyze, e.g. ["GW190412_TaylorF2"]

    Returns:
        pd.DataFrame: Dataframe containing the events and the JS divergences for all parameters
    """
    
    table = {"Parameter": parameters_str}
    MY_LENGTH = nb_round + 2 # number of characters, including the "0." hence the +2
    
    for event in events_list:
        # Load the JS values
        path = get_path(event)
        with open(path, "rb") as f:
            js_div_dict = pickle.load(f)
            
        # Convert and fetch the highest value's index
        js_div_values = np.array(list(js_div_dict.values()))
        js_div_values = np.round(js_div_values, nb_round)
        max_index = np.argmax(js_div_values)
        
        # Get their string representations
        js_div = [str(value) for value in js_div_values]
        for i, value in enumerate(js_div):
            length_difference = MY_LENGTH - len(value)
            if length_difference > 0:
                js_div[i] = value + filler_str * (MY_LENGTH - len(value))
        
        js_div = [f"${value}$" for value in js_div]
        
        # Make the highest value bold
        js_div[max_index] = "$\\bm{{{}}}$".format(str(js_div_values[max_index]))
        table[event] = js_div
    
    print("table")
    print(table)
    
    table = pd.DataFrame.from_dict(table)
    table.rename(columns={'GW170817_TaylorF2': 'TF2', 
                       'GW170817_NRTidalv2': 'NRTv2',
                       'GW190425_TaylorF2': 'TF2', 
                       'GW190425_NRTidalv2': 'NRTv2',}, inplace=True)
    return table

In [152]:
def my_to_latex(df: pd.DataFrame, 
                extra_table_header: str = None,
                column_format: str = None,
                only_tabular: bool = True,
                use_hline: bool = True) -> str:
    """ 
    Own auxiliary function to create latex code of tables in pandas DF format.
    """
    
    n_cols = len(df.columns)
    
    # Column format
    if column_format is None:
        column_format = "l " + " c " * (n_cols - 1)
    
    result = df.style \
        .hide(axis="index")\
        .to_latex(position_float="centering", hrules=True, column_format = column_format)
        
    # Insert "\\midrule" after the row with "ref" in the second column
    latex_table_lines = result.split('\n')
    # Find index with ref isn it:
    insert_idx = 0
    for i in range(len(latex_table_lines)):
        line = latex_table_lines[i]
        if "ref" in line:
            insert_idx = i
            break
        
    latex_table_lines.insert(insert_idx + 1, "\\midrule")
    result = '\n'.join(latex_table_lines)
    
    # Add extra line to table header, eg to group different columns
    if extra_table_header is not None:
        result = result.replace("\\toprule", "\\toprule\n" + extra_table_header)
        
    # Remove table env
    if only_tabular:
        latex_table_lines = result.split('\n')
        latex_table_lines = [line for line in latex_table_lines if "{table}" not in line and "centering" not in line]
        result = '\n'.join(latex_table_lines)
        
    if use_hline:
        result = result.replace("\\bottomrule", "\hline\hline")
        result = result.replace("\\toprule", "\hline\hline")
        result = result.replace("\midrule", "")
    
    return result 

## Make the JS divergence table

In [153]:
events_list = ["GW170817_TaylorF2",
               "GW170817_NRTidalv2",
               "GW190425_TaylorF2",
               "GW190425_NRTidalv2"]

column_names = []

In [154]:
table = get_js_table(events_list)
table

table
{'Parameter': ['$\\mathcal{M}_c$', '$q$', '$\\chi_1$', '$\\chi_2$', '$\\Lambda_1$', '$\\Lambda_2$', '$d_L$', '$\\phi_c$', '$\\cos \\iota$', '$\\psi$', '$\\alpha$', '$\\sin \\delta$'], 'GW170817_TaylorF2': ['$\\bm{0.256207}$', '$0.047711$', '$0.034082$', '$0.010732$', '$0.026499$', '$0.047872$', '$0.081147$', '$0.028113$', '$0.032104$', '$0.073289$', '$0.141645$', '$0.079993$'], 'GW170817_NRTidalv2': ['$0.000919$', '$0.004092$', '$0.001766$', '$0.000846$', '$0.001601$', '$0.001492$', '$\\bm{0.018794}$', '$0.013442$', '$0.011208$', '$0.016563$', '$0.000722$', '$0.000444$'], 'GW190425_TaylorF2': ['$0.000805$', '$0.034010$', '$0.006970$', '$0.007001$', '$0.001268$', '$0.000996$', '$\\bm{0.086857}$', '$0.001121$', '$0.050104$', '$0.000414$', '$0.024573$', '$0.010900$'], 'GW190425_NRTidalv2': ['$0.001706$', '$0.002051$', '$0.003344$', '$0.002108$', '$0.000543$', '$0.002478$', '$0.001995$', '$0.004246$', '$0.003659$', '$0.002586$', '$\\bm{0.00852}$', '$0.006396$']}


,Parameter,TF2,NRTv2,TF2,NRTv2
0,$\mathcal{M}_c$,$\bm{0.256207}$,$0.000919$,$0.000805$,$0.001706$
1,$q$,$0.047711$,$0.004092$,$0.034010$,$0.002051$
2,$\chi_1$,$0.034082$,$0.001766$,$0.006970$,$0.003344$
3,$\chi_2$,$0.010732$,$0.000846$,$0.007001$,$0.002108$
4,$\Lambda_1$,$0.026499$,$0.001601$,$0.001268$,$0.000543$
5,$\Lambda_2$,$0.047872$,$0.001492$,$0.000996$,$0.002478$
6,$d_L$,$0.081147$,$\bm{0.018794}$,$\bm{0.086857}$,$0.001995$
7,$\phi_c$,$0.028113$,$0.013442$,$0.001121$,$0.004246$
8,$\cos \iota$,$0.032104$,$0.011208$,$0.050104$,$0.003659$
9,$\psi$,$0.073289$,$0.016563$,$0.000414$,$0.002586$


In [159]:
# TODO: this is when all runs are finished
# table_header = "\\begin{{tabular}}{{l c c c c}}\n \hline \hline\n & \\multicolumn{{2}}{{ c }}{{GW170817}} &  \\multicolumn{{2}}{{ c }}{{GW170817}} \\\\\n"

# table_header = "\\begin{{tabular}}{{l c c c c}}\n \hline \hline\n & \\multicolumn{{2}}{{ c }}{{GW170817}} \\\\\n"
table_header = "& \multicolumn{2}{ c }{GW170817} & \multicolumn{2}{ c }{GW190425} \\\\\n"

In [160]:
table_str = my_to_latex(table, extra_table_header=table_header)

print(table_str)


\begin{tabular}{l  c  c  c  c}
\hline\hline
& \multicolumn{2}{ c }{GW170817} & \multicolumn{2}{ c }{GW190425} \\

Parameter & TF2 & NRTv2 & TF2 & NRTv2 \\

$\mathcal{M}_c$ & $\bm{0.256207}$ & $0.000919$ & $0.000805$ & $0.001706$ \\
$q$ & $0.047711$ & $0.004092$ & $0.034010$ & $0.002051$ \\
$\chi_1$ & $0.034082$ & $0.001766$ & $0.006970$ & $0.003344$ \\
$\chi_2$ & $0.010732$ & $0.000846$ & $0.007001$ & $0.002108$ \\
$\Lambda_1$ & $0.026499$ & $0.001601$ & $0.001268$ & $0.000543$ \\
$\Lambda_2$ & $0.047872$ & $0.001492$ & $0.000996$ & $0.002478$ \\
$d_L$ & $0.081147$ & $\bm{0.018794}$ & $\bm{0.086857}$ & $0.001995$ \\
$\phi_c$ & $0.028113$ & $0.013442$ & $0.001121$ & $0.004246$ \\
$\cos \iota$ & $0.032104$ & $0.011208$ & $0.050104$ & $0.003659$ \\
$\psi$ & $0.073289$ & $0.016563$ & $0.000414$ & $0.002586$ \\
$\alpha$ & $0.141645$ & $0.000722$ & $0.024573$ & $\bm{0.00852}$ \\
$\sin \delta$ & $0.079993$ & $0.000444$ & $0.010900$ & $0.006396$ \\
\hline\hline
\end{tabular}

